In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import PIL

from torch.utils.data import (
    ConcatDataset,
    DataLoader
)

device = "cuda"

In [2]:
import os
from typing import Tuple

import numpy as np
from PIL import Image
from torch.utils.data import Dataset

def get_boxes(arr: np.ndarray) -> dict[str, torch.Tensor]:
    boxes = []
    lbls = np.delete(np.unique(arr), 0)
    for i in lbls:
        xcoord, ycoord = np.where(arr == i)
        boxes.append([xcoord.min(), ycoord.min(), xcoord.max(), ycoord.max()])
    return {
        "boxes": torch.FloatTensor(np.array(boxes)),
        "labels": lbls,
        "raw_mask": arr
    }

class CellposeDataset(Dataset):
    images: list[np.ndarray]
    masks: list[dict[str, torch.Tensor]]

    def __init__(self, path: str):
        n_files = len(os.listdir(path))
        assert n_files % 2 == 0

        count = n_files // 2

        print("Loading images")
        self.images = list(map(
            lambda idx: np.array(Image.open("{}/{:03}_img.png".format(
                path, idx
            ))).transpose((2,0,1)) / 255.,
            range(count)
        ))

        print("Loading masks")
        self.masks = list(map(
            lambda idx: get_boxes(
                np.array(Image.open("{}/{:03}_masks.png".format(
                    path, idx
                )))
            ),
            range(count)
        ))
    
    def __getitem__(self,idx: int):
        img = self.images[idx]
        mask_info = self.masks[idx].copy()
        # We build the mask now because it consume too much memory
        masks = [
            (mask_info["raw_mask"] == i).astype(int)
            for i in mask_info["labels"]
        ]
        mask_info["boxes"] = mask_info["boxes"].to(device)
        mask_info["masks"] = torch.IntTensor(masks).to(device)
        mask_info["labels"] = torch.IntTensor(np.ones(mask_info["labels"].shape)).to(device)
        return img, mask_info

    def __len__(self) -> int:
        return len(self.images)


In [3]:
test_dt = CellposeDataset("./test")
train_dt = CellposeDataset("./train")

Loading images
Loading masks
Loading images
Loading masks


In [12]:
model = torchvision.models.detection.maskrcnn_resnet50_fpn(num_classes = 2).to(device)

In [13]:
model.eval()
model(torch.tensor([test_dt[0][0]], dtype=torch.float).to(device))

NotImplementedError: Could not run 'torchvision::nms' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'torchvision::nms' is only available for these backends: [Meta, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMTIA, AutogradMeta, Tracer, AutocastCPU, AutocastMTIA, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

Meta: registered at /dev/null:214 [kernel]
BackendS